# a

In [1]:
import numpy as np

In [2]:
def preconditioned_conjugate_gradient_method(
    A_mat: np.ndarray,
    b_vec: np.ndarray,
    M_mat: np.ndarray,
    x_vec: np.ndarray,
    epsilon: np.float64,
) -> tuple[np.ndarray, int]:
    """Implement of the alg 8.8 in the notes."""
    r = b_vec - A_mat @ x_vec
    z = np.linalg.solve(M_mat, r)
    p = z.copy()
    k = 0
    while np.sqrt(np.inner(z.flatten(), r.flatten())) > epsilon:
        k += 1
        s = A_mat @ p
        v = np.inner(z.flatten(), r.flatten()) / np.inner(p.flatten(), s.flatten())
        x_vec = x_vec + v * p
        r_old = r.copy()
        r = r - v * s
        z_old = z.copy()
        z = np.linalg.solve(M_mat, r)
        mu = np.inner(z.flatten(), r.flatten()) / np.inner(
            z_old.flatten(), r_old.flatten()
        )
        p = z + mu * p

    return x_vec, k

In [3]:
def gen_A(n: int) -> np.ndarray:
    """Generate the matrix A of the linear system of equations Au=b."""
    n = n - 1
    A = np.zeros((n, n))
    for i in range(n):
        A[i, i] = 2
        if i > 0:
            A[i, i - 1] = -1
        if i < n - 1:
            A[i, i + 1] = -1
    return A * ((n + 1) ** 2)


gen_A(5)

array([[ 50., -25.,   0.,   0.],
       [-25.,  50., -25.,   0.],
       [  0., -25.,  50., -25.],
       [  0.,   0., -25.,  50.]])

In [4]:
def gen_b(n: int) -> np.ndarray:
    """Generate the vector b of the linear system of equations Au=b."""
    b = np.zeros(n - 1)
    h = np.float64(1 / n)

    def f(x: np.float64) -> np.float64:
        """The function f in the linear system of equations Au=p."""
        return -6 * np.pi * np.cos(3 * np.pi * x) + 9 * np.pi**2 * x * np.sin(
            3 * np.pi * x
        )

    for i in range(n - 1):
        b[i] = f((i + 1) * h)
    return b.reshape(-1, 1)


gen_b(5)

array([[ 22.72062596],
       [ -5.63473741],
       [-46.57613381],
       [ 61.75833825]])

In [5]:
n = [8, 16, 32, 64, 128, 256, 512, 1024]
k_ls = [[] for _ in range(4)]
for i in n:
    A = gen_A(i)
    b = gen_b(i)
    x = np.zeros((i - 1, 1))
    # no preconditioner
    M = np.eye(i - 1)
    _, k = preconditioned_conjugate_gradient_method(A, b, M, x, 1e-10)
    k_ls[0].append(k)
    # Jacobi Preconditioner
    M = np.diag(np.diag(A))
    _, k = preconditioned_conjugate_gradient_method(A, b, M, x, 1e-10)
    k_ls[1].append(k)
    # SSOR Preconditioner
    L = np.tril(A)
    D = np.diag(np.diag(A))
    U = np.triu(A)
    M = (D + L) @ np.linalg.inv(D) @ (D + U)
    _, k = preconditioned_conjugate_gradient_method(A, b, M, x, 1e-10)
    k_ls[2].append(k)
    # L Preconditioner
    L = np.linalg.cholesky(A)
    M = L.T @ L
    _, k = preconditioned_conjugate_gradient_method(A, b, M, x, 1e-10)
    k_ls[3].append(k)

display(k_ls)

[[4, 8, 16, 32, 64, 128, 256, 513],
 [4, 8, 16, 32, 64, 128, 256, 512],
 [7, 15, 28, 49, 89, 168, 300, 467],
 [7, 11, 12, 13, 15, 16, 17, 18]]